# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [8]:
#Your code here
import pandas as pd
df = pd.read_csv("homepage_actions.csv")
print(df.shape)
print(df.info())
df.head()

(8188, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [9]:
# Viewers
print("click/views:", df["action"].value_counts())
# Overlap check
print("group counts:", df["group"].value_counts())

click/views: view     6328
click    1860
Name: action, dtype: int64
group counts: control       4264
experiment    3924
Name: group, dtype: int64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [30]:
"""
Ho-- The homepage for both groups are indifferent.
Ha-- The homepage for the experimental design is more effective compared to the 
     control group.
"""

'\nHo-- The homepage for both groups are indifferent.\nHa-- The homepage for the experimental design is more effective compared to the \n     control group.\n'

In [22]:
df["count"] = 1

#Your code here
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value = 0)
experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value = 0)
print("control:", control.click)
print("experiment:", experiment.click)

control: id
182994    1.0
183089    0.0
183248    1.0
183515    0.0
183524    0.0
         ... 
936786    0.0
937003    0.0
937073    0.0
937108    0.0
937217    1.0
Name: click, Length: 3332, dtype: float64
experiment: id
182988    0.0
183136    0.0
183141    1.0
183283    0.0
183389    0.0
         ... 
935382    0.0
935576    0.0
935742    1.0
936129    0.0
937139    1.0
Name: click, Length: 2996, dtype: float64


In [23]:
# Performing the test
import scipy.stats as stats
stats.ttest_ind(control.click, experiment.click)

Ttest_indResult(statistic=-2.6195696844542207, pvalue=0.008825098914958288)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [24]:
#Your code here
control_rate = control.click.mean()
clicks_exp = control_rate*len(experiment)
print(clicks_exp)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [28]:
import numpy as np
# Your code here
n = len(experiment)
p = control_rate
var = n * p* (1-p)
std = np.sqrt(var)
print(std)
# Z-score
experiment_clicks = experiment.click.sum()
z_score = (experiment_clicks - clicks_exp)/std
z_score

24.568547907005815


3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [29]:
#Your code here
p_value = stats.norm.sf(z_score)
p_value

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p_value matches the previous one, though it's slightly lower.
These results would lead to the rejection of the null hypothesis. The experimental homepage has a more effective design than that of the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.